In [21]:
# Import libraries
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from urllib.request import urlopen
from urllib.request import Request
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Parameters 
n = 3 #the # of article headlines displayed per ticker
tickers = ['AAPL', 'TSLA', 'AMZN']

In [22]:
# Get Data
finviz_url = 'https://finviz.com/quote.ashx?t='
news_tables = {}

for ticker in tickers:
    url = finviz_url + ticker
    req = Request(url=url,headers={'user-agent': 'my-app/0.0.1'}) 
    resp = urlopen(req)    
    html = BeautifulSoup(resp, features="lxml")
    news_table = html.find(id='news-table')
    news_tables[ticker] = news_table

try:
    for ticker in tickers:
        df = news_tables[ticker]
        df_tr = df.findAll('tr')
    
        print ('\n')
        print ('Recent News Headlines for {}: '.format(ticker))
        
        for i, table_row in enumerate(df_tr):
            a_text = table_row.a.text
            td_text = table_row.td.text
            td_text = td_text.strip()
            print(a_text,'(',td_text,')')
            if i == n-1:
                break
except KeyError:
    pass



Recent News Headlines for AAPL: 
Dow Jones Futures: Market Correction Extends Losses; Four Stocks In Beat-Up Sector Worth Watching ( Jan-19-22 10:11PM )
Sonos, Yelp Tell White House to Back Big Tech Antitrust Bill ( 09:24PM )
U.S. Antitrust Bill Expanded to Include TikTok, Tencents WeChat ( 09:06PM )


Recent News Headlines for TSLA: 
Gilbert & Cook, Inc. Buys Vanguard FTSE Developed Markets ETF, Vanguard Total Bond Market ... ( Jan-19-22 05:38PM )
Acas, Llc Buys Vanguard Extended Market Index ETF, Vanguard High Dividend Yield Indx ETF, ... ( 05:38PM )
Syverson Strege & Co Buys BTC BlackRock Short Maturity Municipal Bond ETF, Dimensional ... ( 05:38PM )


Recent News Headlines for AMZN: 
Salvus Wealth Management, LLC Buys Verizon Communications Inc, Gartner Inc, Nike Inc, Sells ... ( Jan-19-22 09:38PM )
Sonos, Yelp Tell White House to Back Big Tech Antitrust Bill ( 09:24PM )
U.S. Antitrust Bill Expanded to Include TikTok, Tencents WeChat ( 09:06PM )


In [23]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ym\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [24]:
# Iterate through the news
parsed_news = []
for file_name, news_table in news_tables.items():
    for x in news_table.findAll('tr'):
        text = x.a.get_text() 
        date_scrape = x.td.text.split()

        if len(date_scrape) == 1:
            time = date_scrape[0]
            
        else:
            date = date_scrape[0]
            time = date_scrape[1]

        ticker = file_name.split('_')[0]
        
        parsed_news.append([ticker, date, time, text])

In [25]:
# Sentiment Analysis
analyzer = SentimentIntensityAnalyzer()

columns = ['Ticker', 'Date', 'Time', 'Headline']
news = pd.DataFrame(parsed_news, columns=columns)
scores = news['Headline'].apply(analyzer.polarity_scores).tolist()

df_scores = pd.DataFrame(scores)
news = news.join(df_scores, rsuffix='_right')

In [26]:
# View Data 
news['Date'] = pd.to_datetime(news.Date).dt.date

unique_ticker = news['Ticker'].unique().tolist()
news_dict = {name: news.loc[news['Ticker'] == name] for name in unique_ticker}

values = []
for ticker in tickers: 
    dataframe = news_dict[ticker]
    dataframe = dataframe.set_index('Ticker')
    dataframe = dataframe.drop(columns = ['Headline'])
    print ('\n')
    print (dataframe.head())
    
    mean = round(dataframe['compound'].mean(), 2)
    values.append(mean)
    
df = pd.DataFrame(list(zip(tickers, values)), columns =['Ticker', 'Mean Sentiment']) 
df = df.set_index('Ticker')
df = df.sort_values('Mean Sentiment', ascending=False)
print ('\n')
print (df)



              Date     Time    neg    neu    pos  compound
Ticker                                                    
AAPL    2022-01-19  10:11PM  0.158  0.643  0.199   -0.0772
AAPL    2022-01-19  09:24PM  0.000  1.000  0.000    0.0000
AAPL    2022-01-19  09:06PM  0.000  1.000  0.000    0.0000
AAPL    2022-01-19  04:23PM  0.000  1.000  0.000    0.0000
AAPL    2022-01-19  03:20PM  0.000  1.000  0.000    0.0000


              Date     Time  neg  neu  pos  compound
Ticker                                              
TSLA    2022-01-19  05:38PM  0.0  1.0  0.0       0.0
TSLA    2022-01-19  05:38PM  0.0  1.0  0.0       0.0
TSLA    2022-01-19  05:38PM  0.0  1.0  0.0       0.0
TSLA    2022-01-19  05:38PM  0.0  1.0  0.0       0.0
TSLA    2022-01-19  04:24PM  0.0  1.0  0.0       0.0


              Date     Time    neg    neu    pos  compound
Ticker                                                    
AMZN    2022-01-19  09:38PM  0.000  0.802  0.198    0.4939
AMZN    2022-01-19  09:24PM  0.00

In [27]:
# Import libraries
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from urllib.request import urlopen, Request
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Parameters 
n = 3 #the # of article headlines displayed per ticker
tickers = ['AAPL', 'TSLA', 'AMZN']

# Get Data
finwiz_url = 'https://finviz.com/quote.ashx?t='
news_tables = {}

for ticker in tickers:
    url = finwiz_url + ticker
    req = Request(url=url,headers={'user-agent': 'my-app/0.0.1'}) 
    resp = urlopen(req)    
    html = BeautifulSoup(resp, features="lxml")
    news_table = html.find(id='news-table')
    news_tables[ticker] = news_table

try:
    for ticker in tickers:
        df = news_tables[ticker]
        df_tr = df.findAll('tr')
    
        print ('\n')
        print ('Recent News Headlines for {}: '.format(ticker))
        
        for i, table_row in enumerate(df_tr):
            a_text = table_row.a.text
            td_text = table_row.td.text
            td_text = td_text.strip()
            print(a_text,'(',td_text,')')
            if i == n-1:
                break
except KeyError:
    pass

# Iterate through the news
parsed_news = []
for file_name, news_table in news_tables.items():
    for x in news_table.findAll('tr'):
        text = x.a.get_text() 
        date_scrape = x.td.text.split()

        if len(date_scrape) == 1:
            time = date_scrape[0]
            
        else:
            date = date_scrape[0]
            time = date_scrape[1]

        ticker = file_name.split('_')[0]
        
        parsed_news.append([ticker, date, time, text])
        
# Sentiment Analysis
analyzer = SentimentIntensityAnalyzer()

columns = ['Ticker', 'Date', 'Time', 'Headline']
news = pd.DataFrame(parsed_news, columns=columns)
scores = news['Headline'].apply(analyzer.polarity_scores).tolist()

df_scores = pd.DataFrame(scores)
news = news.join(df_scores, rsuffix='_right')


# View Data 
news['Date'] = pd.to_datetime(news.Date).dt.date

unique_ticker = news['Ticker'].unique().tolist()
news_dict = {name: news.loc[news['Ticker'] == name] for name in unique_ticker}

values = []
for ticker in tickers: 
    dataframe = news_dict[ticker]
    dataframe = dataframe.set_index('Ticker')
    dataframe = dataframe.drop(columns = ['Headline'])
    print ('\n')
    print (dataframe.head())
    
    mean = round(dataframe['compound'].mean(), 2)
    values.append(mean)
    
df = pd.DataFrame(list(zip(tickers, values)), columns =['Ticker', 'Mean Sentiment']) 
df = df.set_index('Ticker')
df = df.sort_values('Mean Sentiment', ascending=False)
print ('\n')
print (df)



Recent News Headlines for AAPL: 
Dow Jones Futures: Market Correction Extends Losses; Four Stocks In Beat-Up Sector Worth Watching ( Jan-19-22 10:11PM )
Sonos, Yelp Tell White House to Back Big Tech Antitrust Bill ( 09:24PM )
U.S. Antitrust Bill Expanded to Include TikTok, Tencents WeChat ( 09:06PM )


Recent News Headlines for TSLA: 
Gilbert & Cook, Inc. Buys Vanguard FTSE Developed Markets ETF, Vanguard Total Bond Market ... ( Jan-19-22 05:38PM )
Acas, Llc Buys Vanguard Extended Market Index ETF, Vanguard High Dividend Yield Indx ETF, ... ( 05:38PM )
Syverson Strege & Co Buys BTC BlackRock Short Maturity Municipal Bond ETF, Dimensional ... ( 05:38PM )


Recent News Headlines for AMZN: 
Salvus Wealth Management, LLC Buys Verizon Communications Inc, Gartner Inc, Nike Inc, Sells ... ( Jan-19-22 09:38PM )
Sonos, Yelp Tell White House to Back Big Tech Antitrust Bill ( 09:24PM )
U.S. Antitrust Bill Expanded to Include TikTok, Tencents WeChat ( 09:06PM )


              Date     Time    n